## Sketchbook

Simple sketchbook where we run functions that we will modify in the main code (tfbs.ipynb) to check and see if it does what we expect it to.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
from numpy.polynomial.legendre import leggauss
from scipy.special import sph_harm
from sympy.physics.quantum.cg import CG, Wigner9j, Wigner6j

Some functions we need to test the permutation operator.

Some things we need to note:
- $p, q$ are equivalent to $p_{12}, p_3$ respectively.
- Here $p_3^\prime$ is only in the z-direction, so the magnetic component is zero (orbital ang mom only
in one direction), i.e. $m_{3} = 0 \implies m_{12}=M$. But $p_3$ is in both x and z direction and so can vary (defined via $\mu$, $M-\mu$ for $m_{12}, m_3$ resp.) 

Below is the transformation method to transform the grid points for the Gauss-Legendre quadrature (needed to get grid and weights for integration)

In [2]:
def _trns(np1,np2,pa,pb,pc):
  """Auxilliary method that provides transformed Gauss-Legendre grid points and integration weights.

     This is using a hyperbolic trafo shown in the lecture. 
     Parameter: 
     np1 --  grid points in ]0,pb[
     np2 --  grid points are distributed in ]pb,pc[ using a linear trafo

     pa  -- half of np1 points are in interval [0,pa]
     pb  -- interval boundary as defined above 
     pc  -- upper integration boundary """ 

  x1grid,x1weight=leggauss(np1)
  x2grid,x2weight=leggauss(np2)

  npoints = np1 + np2

  # trafo (1.+X) / (1./P1-(1./P1-2./P2)*X) for first interval 
  p1grid=(1.+x1grid) / (1./pa-(1./pa-2./pb)*x1grid)
  p1weight=(2.0/pa-2.0/pb)*x1weight / (1./pa-(1./pa-2./pb)*x1grid)**2

  # linear trafo 
  p2grid=(pc+pb)/2.0 + (pc-pb)/2.0*x2grid
  p2weight=(pc-pb)/2.0*x2weight

  pgrid=np.empty((npoints),dtype=np.double)
  pweight=np.empty((npoints),dtype=np.double)

  pgrid = np.concatenate((p1grid, p2grid), axis=None)
  pweight = np.concatenate((p1weight, p2weight), axis=None)

  return pgrid,pweight 

Below is auxiliary function that converts three component vector into spherical coordinates of $(p, \theta, \phi)$ (ex. $\vec{p_{12}} = (p_{12}, \theta_{12}, \phi_{12})$)

In [3]:
def _angle(px,py,pz):
    """Auxiliary routine to determine magnitude, phi, and theta of three component vector. 

       Parameters:
       px,py,pz -- cartesian components of a vector 

       returns magntitude, theta and phi angles.
    """

    pmag=np.sqrt(px**2+py**2+pz**2)
    theta=np.where(pmag!=0.0,np.arccos(pz/pmag),0.0)

    phi=theta # copy shape of theta to phi 
    phi=1.5*m.pi # and set to constant value

    # prepare bool arrays for px,py > 0 < 0  with shape of phi 

    pxgt0=(phi==0)  # copy shape
    pxgt0=(px>0)    # test 

    pxlt0=(phi==0)  # copy shape
    pxlt0=(px<0)    # test 

    pxeq0=(phi==0)  # copy shape
    pxeq0=(px==0)   # test 

    pygt0=(phi==0)  # copy shape
    pygt0=(py>0)    # test 

    np.where(pxgt0 & pygt0,np.arctan(py/px),phi)
    np.where(pxgt0 & np.invert(pxgt0),2*m.pi-np.arctan(-py/px),phi)
    np.where(pxlt0 & pygt0,m.pi-np.arctan(-py/px),phi)
    np.where(pxlt0 & np.invert(pygt0),m.pi+np.arctan(py/px),phi)
    np.where(pxeq0 & pygt0,0.5*m.pi,phi)

    return pmag,theta,phi   

below is auxiliary function for converting index $l, m$ into a combined index.

In [4]:
def _lmindx(l,m):
    """Combined unique index for l and m.
    
        Nice trick: since quantum numbers lm are linked to each other, this combined 
        index allows one to store the results depending on lm using the memory more efficiently. 
    """        
    return l**2+l+m

Here below we test the execution of the permutation operator.

In [5]:
# some initial parameters set by ThreeBody (inherited from TwoBodyTMat) class

# gridpoints for momentum transfer
nq1=20
nq2=10
nqpoints = nq1 + nq2

# grid points for x (cos of angle between p3 and p3')
nx = 12

# lmax=0  # max two-body angular momentum
# bl=1  # total orbital angular momentum

# momentum grid interval boundaries
qa=1.0
qb=5.0
qc=20.0

xp,xw = leggauss(nx)  # grid points for permutation op

# # spin states
# smax = 0.5   # max value for spin quantum number
# s3 = 0.5   # s3 fixed to be 1/2
# bs = 0.5   # total spin S

# # total states
# bj = bl + bs   # total angular momentum J = L + S
# jmax = lmax + smax   # max total angular momentum jmax = lmax + smax

# # creating the alpha dictionary
# lammax=lmax+bl  # max value for l3
# jlammax = jmax + bj  # max value for j3    


# bl=bl  # total orbital angular momentum L
# lammax=lmax+bl  # max value for l3
# qnalpha=[]
# alpha=0
# alphal = 0   # alpha for orbital angular momentum

# # # creating dictionaries for alpha, that contains l12, l3, L, s12, s3, 
# # for l in range(lmax+1):
# #     if(l%2==0):   # take only symmetric pw (Pauli)  
# #         for lam in range(abs(l-bl),l+bl+1):
# #             alphal += 1
# #             for s12 in np.arange(0.5, smax+1, 1):   # iterate over possible spin states
# #                 j = l + s12
# #                 j3 = lam + s3
# #                 qnalpha.append({"alpha":alpha, "alphal":alphal, "l":l,"lam":lam, "s": s12, "s3":s3, "j":j, 
# #                              "j3":j3, "bl":bl, "bs":bs, "bj":bj})
# #                 alpha+=1

# for l in range(lmax+1):
#   if(l%2==0):   # take only symmetric pw (Pauli)  
#    for lam in range(abs(l-bl),l+bl+1):
#      qnalpha.append({"alpha":alpha,"l":l,"lam":lam,"bl":bl})
#      alpha+=1
# nalpha=len(qnalpha)

# nalphal = int(alphal)

# creating momentum transfer grid
qgrid,qweight = _trns(nq1,nq2,qa,qb,qc)

Below is the formulation of the permutation operator.

- `pip, pi` are $\pi_{12}^\prime$, $\pi_{12}$ respectively.
- `thetapp` is the arccosine of $x$ (angle between p3 and p3')
- `theta, thetap` are the direction of p12 and p3 resp.

In [6]:
# prepare shifted momenta and angles for the symmetric permutation 

pip=np.empty((nqpoints, nqpoints, nx),dtype=np.double)        
pi=np.empty(( nqpoints, nqpoints, nx),dtype=np.double)        

thetap=np.empty(( nqpoints, nqpoints, nx),dtype=np.double)
theta=np.empty(( nqpoints, nqpoints, nx),dtype=np.double)
thetapp=np.empty(( nx),dtype=np.double)

# construct direction and magnitude vectors for p12, p3, x
for ix in range( nx):
  xval= xp[ix] 
  thetapp[ix]=np.arccos(xval)
  for jq in range( nqpoints):
    qpval= qgrid[jq]
    for iq in range( nqpoints):
      qval= qgrid[iq]

      px=qpval*np.sqrt(1.0-xval**2)
      py=0.0
      pz=0.5*qval+qpval*xval 
      pi[iq,jq,ix],theta[iq,jq,ix],phi= _angle(px,py,pz)

      px=-0.5*qpval*np.sqrt(1.0-xval**2)
      py=0.0
      pz=-qval-0.5*qpval*xval 
      pip[iq,jq,ix],thetap[iq,jq,ix],phi= _angle(px,py,pz)



Here the spherical harmonics are constructed.
Since we write the integral of the permutation operator only with respect to $x = \hat{p_3}$, the spherical harmonics are written in this fashion as well. (Lecture 9, page 6, line 3). This is contained within the definition of the permutation operator $G_{\alpha^\prime, \alpha}$

So here we find the spherical harmonics $y_{l,\mu}(\hat{-q-0.5q*p}), y_{l,\mu}(\hat{q+0.5q*p})$ and $y_{\lambda, \mu}(\hat{qp})$, where $(l, \mu) = (l_{12}, m)$ and $(\lambda, \mu) = (l_3, m)$. 

In [7]:
# # prepare spherical harmonics and store based on lmindx 
# # number of lam,mu und l,mu combinations 
# nlamindx=_lmindx(lammax,lammax)+1
# nlindx=_lmindx(lmax,lmax)+1

# # array for Y_{lam mu}(hat qp) (real is sufficient since phi=0)
# ylam=np.empty((nlamindx,nx),dtype=np.double)
# for lam in range(lammax+1):
#     for mu in range(-lam,lam+1):
#         ylam[_lmindx(lam,mu),:]=np.real(sph_harm(mu,lam, 0, thetapp))


# # array for Y_{l mu}(-q-0.5qp) (real is sufficient since phi=0)
# yl=np.empty((nlindx,nqpoints,nqpoints,nx),dtype=np.double)
# for l in range(lmax+1):
#     for mu in range(-l,l+1):
#         yl[_lmindx(l,mu),:,:,:]=np.real(sph_harm(mu,l, 0, thetap))

# # array for Y*_{l mu}(0.5q+qp) (real is sufficient since phi=0)
# ystarl=np.empty((nlindx,nqpoints,nqpoints,nx),dtype=np.double)
# for l in range(lmax+1):
#     for mu in range(-l,l+1):
#         ystarl[_lmindx(l,mu),:,:,:]=np.real(sph_harm(mu,l, 0, theta))

Here then we evaluate the CG coefficients contained in the coupled spherical harmonics.

The primed coordinates have $(M, 0, M)$ due to how $p_3$ is defined.
The unprimed coordinates have a x-dependence so its $(\mu, M-\mu, M)$.
They also share the same total magnetic quantum number and the orbital angular momentum too (why same l12, l3?).


We need to modify the definition of `alpha` so that it includes the spin components. This means modifying all of the code down below so that the for loop iterations are properly dealt with.

In [7]:
# bl=bl  # total orbital angular momentum L
# lammax=lmax+bl  # max value for l3


lmax=0  # max two-body angular momentum
# bl=0  # total orbital angular momentum
bj=1
lammax=0  # max value for l3

# spin states
smax = 0.5   # max value for spin quantum number
s3 = 0.5   # s3 fixed to be 1/2
# bs = smax * 2 + s3

# total states
# bj = bl + bs   # total angular momentum J = L + S
# jmax = lmax + smax   # max total angular momentum jmax = lmax + smax
# jlammax = jmax + bj  # max value for j3

# creating the alpha dictionary
qnalpha=[]
alpha=0
alphal = 0   # alpha for orbital angular momentum

# # creating dictionaries for alpha, that contains l12, l3, L, s12, s3, 
# for l12 in range(lmax+1):
#     if(l12%2==0):   # take only symmetric pw (Pauli)  
#         for lam in range(abs(l12-bl),l12+bl+1):
#             # min spin number is 0.5 since fermionic
#             for s12 in np.arange(0, 2, 1):   # iterate over possible combination of spin states
# #                 s12 = 2*s
#                 j12 = l12 + s12
#                 j3 = lam + s3
#                 bs = s12 + s3
#                 bj = bs + bl
#                 qnalpha.append({"alpha":alpha, "alphal":alphal, "l":l12,"lam":lam, "s": s12, "s3":s3, "j":j12, 
#                              "j3":j3, "bl":bl, "bs":bs, "bj":bj})
#                 alpha+=1
#             alphal += 1

# nalpha=len(qnalpha)
# print(nalpha)
# nalphal = int(alphal)

# print(nalphal)

# cg=np.zeros((nalphal,2*bl+1),dtype=np.double)
# cgp=np.zeros((nalphal,2*lmax+1,2*bl+1),dtype=np.double)

# for qnset in qnalpha:  # go through allowed l,lam combinations
#     for bm in range(-bl,bl+1):
#         cg[qnset["alphal"],bm+bl]=float(CG(qnset["l"],bm,qnset["lam"],0,bl,bm).doit())
#     for mu in range(-qnset["l"],qnset["l"]+1):
#         cgp[qnset["alphal"],mu+qnset["l"],bm+bl]=float(CG(qnset["l"],mu,qnset["lam"],bm-mu,bl,bm).doit())
        
        
        

# for l in range(lmax+1):
#   if(l%2==0):   # take only symmetric pw (Pauli)  
#    for lam in range(abs(l-bl),l+bl+1):
#      qnalpha.append({"alpha":alpha,"l":l,"lam":lam,"bl":bl})
#      alpha+=1
    
nalpha=len(qnalpha)
nalphal = int(alphal)

# print(nalphal)
    
cg=np.zeros((nalpha,2*bl+1),dtype=np.double)
cgp=np.zeros((nalpha,2*lmax+1,2*bl+1),dtype=np.double)

for qnset in qnalpha:  # go through allowed l,lam combinations
    for bm in range(-bl,bl+1):
        cg[qnset["alpha"],bm+bl]=float(CG(qnset["l"],bm,qnset["lam"],0,bl,bm).doit())
    for mu in range(-qnset["l"],qnset["l"]+1):
        cgp[qnset["alpha"],mu+qnset["l"],bm+bl]=float(CG(qnset["l"],mu,qnset["lam"],bm-mu,bl,bm).doit())
        
        
# this should give CG coefficients corresponding to each spin component.

In [7]:
lmax = 1
lammax = 1   # max for orbital angular momentum for 3rd particle
# bl = 0   # total orbital angular momentum

# spin / total ang mom / isospin state parameters
# smax, jmax, n_isospins are all defined in TwoBodyTmat object
s3 = 0.5   # always fixed to be spin-1/2 particles
smax = 1
bs = 1.5
t3 = 0.5   # isospin state is always 1/2 for proton / neutron
n_iso = 2
jmax = 2
j3max =  lammax +  s3
# bj = bl + bs
bj = 1.5
blmax = int(bj-0.5)



qnalpha=[]
#         alpha=0
#         for l in range(lmax+1):
#               if(l%2==0):   # take only symmetric pw (Pauli)  
#                for lam in range(abs(l-bl),l+bl+1):
#                   qnalpha.append({"alpha":alpha,"l":l,"lam":lam,"bl":bl})
#                  alpha+=1
alpha=0
for bl in range(blmax+1):
    bs=bj-bl
    for l in range(lmax+1):
        for s in range(smax+1):
            for j in range(jmax+1):
                if(l%2==0):   # take only symmetric pw (Pauli)  
                    for lam in range(abs(l-bl),l+bl+1):
            #             for lam in range(-lammax,lammax+1):
                            # print(lam)
                        for j3 in np.arange(abs(j-bj),j+bj+1):
                            # print(l,j,s,t)
                            qnalpha.append({"alpha":alpha,"l":l,"lam":lam,"bl":bl,
                                            "s":s, "s3": s3, "bs":bs,
                                            "j":j, "j3":j3, "bj": bj,
                                            "t":1, "t3": t3, "bt":1.5
                                            })
                            alpha+=1
nalpha=len( qnalpha)

In [8]:
# prepare spherical harmonics and store based on lmindx 
# number of lam,mu und l,mu combinations 

# we need to modify the index for the spherical harmonics so that the 
# length of the array matches those that include spin
nlamindx=_lmindx(lammax,lammax)+1
nlindx=_lmindx(lmax,lmax)+1

print(nlamindx, nlindx)

# array for Y_{lam mu}(hat qp) (real is sufficient since phi=0)
ylam=np.empty((nlamindx, smax+1,  jmax+1, n_iso,  nx),dtype=np.double)
for lam in range(lammax+1):
    for mu in range(-lam,lam+1):
        ylam[_lmindx(lam,mu),:,:,:,:]=np.real(sph_harm(mu,lam, 0, thetapp))


# array for Y_{l mu}(-q-0.5qp) (real is sufficient since phi=0)
yl=np.empty((nlindx, smax+1,  jmax+1, n_iso,nqpoints,nqpoints,nx),dtype=np.double)
for l in range(lmax+1):
    for mu in range(-l,l+1):
#         print(np.real(sph_harm(mu,l, 0, thetap)))
        yl[_lmindx(l,mu),:,:,:,:,:,:]=np.real(sph_harm(mu,l, 0, thetap))

# array for Y*_{l mu}(0.5q+qp) (real is sufficient since phi=0)
ystarl=np.empty((nlindx, smax+1,  jmax+1, n_iso,nqpoints,nqpoints,nx),dtype=np.double)
for l in range(lmax+1):
    for mu in range(-l,l+1):
#         print(np.real(sph_harm(mu,l, 0, theta)))
        ystarl[_lmindx(l,mu),:,:,:,:,:,:]=np.real(sph_harm(mu,l, 0, theta))

4 4


In [9]:
print(np.nonzero(ystarl))

(array([0, 0, 0, ..., 3, 3, 3], dtype=int64), array([0, 0, 0, ..., 1, 1, 1], dtype=int64), array([0, 0, 0, ..., 2, 2, 2], dtype=int64), array([0, 0, 0, ..., 1, 1, 1], dtype=int64), array([ 0,  0,  0, ..., 29, 29, 29], dtype=int64), array([ 0,  0,  0, ..., 29, 29, 29], dtype=int64), array([ 0,  1,  2, ...,  9, 10, 11], dtype=int64))


In [10]:
print(ystarl.shape, ylam.shape, yl.shape)
# print((ylam*yl).shape)

(4, 2, 3, 2, 30, 30, 12) (4, 2, 3, 2, 12) (4, 2, 3, 2, 30, 30, 12)


In [11]:
cg=np.zeros((nalpha,2*blmax+1),dtype=np.double)
cgp=np.zeros((nalpha,2*lmax+1,2*blmax+1),dtype=np.double)

for qnset in qnalpha:  # go through allowed l,lam combinations
    bl=qnset["bl"]
    for bm in range(-bl,bl+1):
        # print(bm)
        cg[qnset["alpha"],bm+bl]=float(CG(qnset["l"],bm,qnset["lam"],0,bl,bm).doit())
        for mu in range(-qnset["l"],qnset["l"]+1):
            cgp[qnset["alpha"],mu+qnset["l"],bm+bl]=float(CG(qnset["l"],mu,qnset["lam"],bm-mu,bl,bm).doit())
            # print(cgp[qnset["alpha"],mu+qnset["l"],bm+bl])

Below evaluated the coupled spherical harmonic (evaluates summation of m12 and m3)

In [12]:
# # now we can perform the mu summation for the combination of coupled spherical harmonics 
# ylylam=np.zeros((nalphal,2*bl+1,nqpoints,nqpoints,nx),dtype=np.double)  # coupled sph harmonic array
# for qnset in qnalpha:  # go through allowed l,lam combinations
#     alphapl=qnset["alphal"]
#     l=qnset["l"]
#     lam=qnset["lam"]
#     for bm in range(-bl,bl+1):
#         for mu in range(-l,l+1):
#             lmindx=_lmindx(l,mu)
#             if abs(bm-mu)<=lam:
#                 lamindx=_lmindx(lam,bm-mu)
#                 ylylam[alphapl,bm+bl,:,:,:]+=cgp[alphapl,mu+l,bm+bl]*yl[lmindx,:,:,:]*ylam[lamindx,:]

In [13]:
# print(ylylam[np.nonzero(ylylam)[:]])
# len(ylylam[np.nonzero(ylylam)[:]])

In [14]:
# now we can perform the mu summation for the combination of coupled spherical harmonics 
ylylam=np.zeros((nalpha,2*bl+1,nqpoints,nqpoints,nx),dtype=np.double)  # coupled sph harmonic array
for qnset in qnalpha:  # go through allowed l,lam combinations
    alphap=qnset["alpha"]
    l=qnset["l"]
    lam=qnset["lam"]
    s=qnset["s"]
    s3 = qnset["s3"]
    j=qnset["j"]
    j3=qnset["j3"]
    t=qnset["t"]
    t3=qnset["t3"]
#     print(lam)
    for bm in range(-bl,bl+1):
        for mu in range(-l,l+1):
            lmindx=_lmindx(l,mu)
            if abs(bm-mu)<=lam:
                print(abs(bm-mu), mu, bm)
                lamindx=_lmindx(lam,bm-mu)
#                 print(lamindx)
#                 print(ylam[lamindx,s,j,t,:])
#                 print(bm+bl, alphap)
                ylylam[alphap,bm+bl,:,:,:]+=cgp[alphap,mu+l,bm+bl]*yl[lmindx,s,j,t,:,:,:]*ylam[lamindx,s,j,t,:]
            print(cgp[alphap,mu+l,bm+bl], mu+l, bm+bl)
#             print(np.nonzero(ylylam[alphap,bm+bl,:,:,:]))

1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1.0 0 0
0 0 0
0.0 0 1
0.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 -1
1.0 0 0
0 0 0
1.0 0 1
1 0 1
1.0 0 2
1 0 

In [15]:
print(np.nonzero(ylylam[0,0,:,:,:]))
# len(ylylam[np.nonzero(ylylam)[:]])

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [16]:
# bm summation then gives G 
gfunc=np.zeros((nalpha,nalpha,nqpoints,nqpoints,nx),dtype=np.double)
for qnset in qnalpha:  # go through allowed l,lam combinations
    
    
    alpha=qnset["alpha"]
    l=qnset["l"]
    lam=qnset["lam"]
    s=qnset["s"]
    s3 = qnset["s3"]
    j=qnset["j"]
    j3=qnset["j3"]
    t=qnset["t"]
    t3=qnset["t3"]
    bl = qnset["bl"]
    bs=qnset["bs"]
    bj=qnset["bj"]
    bt=qnset["bt"]
    
    print(l, s, j, t, lam, s3, j3, t3, bl, bs, bj, bt)
    
    for qnsetp in qnalpha:  # go through allowed l,lam combinations
        
        # not sure how this prime stuff works here
        alphap=qnsetp["alpha"]
        sp = s
        jp = j
        j3p = j3
        
        
        
        # no need to code in the summation for L, S since they are the total orbital ang mom and spin,
        # which is fixed for each problem that we do.
        
        # evaluate wigner 9j symbol
        c9j = float(Wigner9j(l, s, j, lam, s3, j3, bl, bs, bj).doit())
        c9jp = float(Wigner9j(l, sp, jp, lam, s3, j3p, bl, bs, bj).doit())
        
        # evaluate spin component -> Wigner 6j symnbols
        c6j = float(Wigner6j(s3, 0.5, sp, s3, bs, s).doit())
        spin_part = (-1)**s * np.sqrt((2*s + 1) * (2*sp + 1)) * c6j
        
        # evaluate coefficients before that 
        coeff = (2*bs + 1) * np.sqrt((2*j+1) * (2*jp+1) * (2*j3+1) + (2*j3p + 1))
        
#         print(c9j, c9jp, c6j, spin_part, coeff)
        
        for bm in range(-bl,bl+1):
            if(abs(bm)<=l):  
                lmindx=_lmindx(l,bm) 
                orbital_part = 8*m.pi**2*np.sqrt((2*lam+1)/(4*m.pi))/(2*bl+1) \
                    *ystarl[lmindx,s,j,t,:,:,:]*ylylam[alphap,bm+bl,:,:,:]
#                 print(bm+bl)
#                 print(np.nonzero(ylylam[alphap,bm+bl,:,:,:]))
                
                gfunc[alpha, alphap, :,:,:] += coeff * c9j * c9jp * spin_part * orbital_part

0 0 0 1 0 0.5 1.5 0.5 0 1.5 1.5 1.5
0 0 1 1 0 0.5 0.5 0.5 0 1.5 1.5 1.5
0 0 1 1 0 0.5 1.5 0.5 0 1.5 1.5 1.5
0 0 1 1 0 0.5 2.5 0.5 0 1.5 1.5 1.5
0 0 2 1 0 0.5 0.5 0.5 0 1.5 1.5 1.5
0 0 2 1 0 0.5 1.5 0.5 0 1.5 1.5 1.5
0 0 2 1 0 0.5 2.5 0.5 0 1.5 1.5 1.5
0 0 2 1 0 0.5 3.5 0.5 0 1.5 1.5 1.5
0 1 0 1 0 0.5 1.5 0.5 0 1.5 1.5 1.5
0 1 1 1 0 0.5 0.5 0.5 0 1.5 1.5 1.5
0 1 1 1 0 0.5 1.5 0.5 0 1.5 1.5 1.5
0 1 1 1 0 0.5 2.5 0.5 0 1.5 1.5 1.5
0 1 2 1 0 0.5 0.5 0.5 0 1.5 1.5 1.5
0 1 2 1 0 0.5 1.5 0.5 0 1.5 1.5 1.5
0 1 2 1 0 0.5 2.5 0.5 0 1.5 1.5 1.5
0 1 2 1 0 0.5 3.5 0.5 0 1.5 1.5 1.5
0 0 0 1 1 0.5 1.5 0.5 1 0.5 1.5 1.5
0 0 1 1 1 0.5 0.5 0.5 1 0.5 1.5 1.5
0 0 1 1 1 0.5 1.5 0.5 1 0.5 1.5 1.5
0 0 1 1 1 0.5 2.5 0.5 1 0.5 1.5 1.5
0 0 2 1 1 0.5 0.5 0.5 1 0.5 1.5 1.5
0 0 2 1 1 0.5 1.5 0.5 1 0.5 1.5 1.5
0 0 2 1 1 0.5 2.5 0.5 1 0.5 1.5 1.5
0 0 2 1 1 0.5 3.5 0.5 1 0.5 1.5 1.5
0 1 0 1 1 0.5 1.5 0.5 1 0.5 1.5 1.5
0 1 1 1 1 0.5 0.5 0.5 1 0.5 1.5 1.5
0 1 1 1 1 0.5 1.5 0.5 1 0.5 1.5 1.5
0 1 1 1 1 0.5 2.5 0.5 1 0.5 

In [17]:
print(gfunc[np.nonzero(gfunc)])
print(gfunc[np.nonzero(gfunc)].shape)

[ 0.08724812  0.19502777  0.29127946 ... -0.01878585 -0.02206072
 -0.02395036]
(691200,)


In [ ]:
# set spline elements based on grid points and shifted momenta 
splpi=Cubherm.spl(pgrid,pi)
splpip=Cubherm.spl(pgrid,pip)

# interpolation fspl=np.sum(spl*fold,axis=1) first axis is pgrid 
# prepare splines multiplied by p**l factors (splalpha also includes the integration weights for q' and x integral)

splalpha=np.empty((npoints*nqpoints*nalpha,nqpoints,nx),dtype=np.double)
splalphap=np.empty((npoints*nqpoints*nalpha,nqpoints,nx),dtype=np.double)

for qnset in qnalpha:  # go through allowed l,lam combinations
    alpha=qnset["alpha"]
    l=qnset["l"]
    for ip in range(npoints): 
    for iq in range(nqpoints):
        indxpmat=npoints*nqpoints*alpha+npoints*iq+ip
        #for jq in range(nqpoints):
        #   splalpha[indxpmat,jq,:]=splpi[ip,iq,jq,:]*(pi[iq,jq,:]/pgrid[ip])**l*xw[:]*qweight[jq]*qgrid[jq]**2
        #   splalphap[indxpmat,jq,:]=splpip[ip,jq,iq,:]*(pip[jq,iq,:]/pgrid[ip])**l
        splalpha[indxpmat,:,:]=np.einsum("ij,j,i->ij",splpi[ip,iq,:,:]*(pi[iq,:,:]/pgrid[ip])**l,xw[:],qweight[:]*qgrid[:]**2)
        splalphap[indxpmat,:,:]=splpip[ip,:,iq,:]*(pip[:,iq,:]/pgrid[ip])**l
    

pmat=np.empty((npoints*nqpoints*nalpha,npoints*nqpoints*nalpha),dtype=np.double)

for qnset in qnalpha:  # go through allowed l,lam combinations
    alpha=qnset["alpha"]
    for qnsetp in qnalpha:  # go through allowed l,lam combinations
    alphap=qnsetp["alpha"]
    for ip in range(npoints): 
        for iq in range(nqpoints):
        indxpmat=npoints*nqpoints*alpha+npoints*iq+ip
        for jp in range(npoints): 
        for jq in range(nqpoints):
        indxpmatp=npoints*nqpoints*alphap+npoints*jq+jp
        pmat[indxpmat,indxpmatp]=np.sum(splalpha[indxpmat,jq,:]
                        *gfunc[alpha,alphap,iq,jq,:]
                        *splalphap[indxpmatp,iq,:])     

Below evaluates the numerical representation of the permutation operator $G_{\alpha^\prime, \alpha}$.
So over here the summation over $M_L$ gives this.

The normalization comes from:
- summation over $M_L$
- the cancellation of the normalization coefficient from the definition of the spherical harmonics in `scipy.special.sph_harm`.

**Modification**
Here we need to add the following:
- the spin components (wigner-6j coefficients)
- the Wigner-9j coefficients

In [91]:
# # bm summation then gives G 
# glfunc=np.zeros((nalphal,nalphal,nqpoints,nqpoints,nx),dtype=np.double)
# for qnset in qnalpha:  # go through allowed l,lam combinations
#     alphal=qnset["alphal"]
#     l=qnset["l"]
#     lam=qnset["lam"]
#     for qnsetp in qnalpha:  # go through allowed l,lam combinations
#         alphapl=qnsetp["alphal"]
#         for bm in range(-bl,bl+1):
#             if(abs(bm)<=l):  
#                 lmindx=_lmindx(l,bm) 
#                 glfunc[alphal,alphapl,:,:,:]+=8*m.pi**2*np.sqrt((2*lam+1)/(4*m.pi))/(2*bl+1) \
#                     *ystarl[lmindx,:,:,:]*ylylam[alphapl,bm+bl,:,:,:]

In [92]:
# print(glfunc[np.nonzero(glfunc)[:]])
# print(len(glfunc[np.nonzero(glfunc)[:]]))
# print(np.argwhere(np.abs(glfunc[np.nonzero(glfunc)[:]]-0.5) > 1e-7))  # check where all values are not 0.5

In [93]:
# # bm summation then gives G 
# gfunc=np.zeros((nalpha,nalpha,nqpoints,nqpoints,nx),dtype=np.double)
# for qnset in qnalpha:  # go through allowed l,lam combinations
#     alpha=qnset["alpha"]
#     l=qnset["l"]
#     lam=qnset["lam"]
#     for qnsetp in qnalpha:  # go through allowed l,lam combinations
#         alphap=qnsetp["alpha"]
#         for bm in range(-bl,bl+1):
#             if(abs(bm)<=l):  
#                 lmindx=_lmindx(l,bm) 
#                 gfunc[alpha,alphap,:,:,:]+=8*m.pi**2*np.sqrt((2*lam+1)/(4*m.pi))/(2*bl+1) \
#                     *ystarl[lmindx,:,:,:]*ylylam[alphap,bm+bl,:,:,:]

In [94]:
# print(gfunc[np.nonzero(gfunc)[:]])
# print(len(gfunc[np.nonzero(gfunc)[:]]))
# print(np.argwhere(np.abs(gfunc[np.nonzero(gfunc)[:]]-0.5) > 1e-7))  # check where all values are not 0.5

Now we perform more summations to add the modifications.

We call the modified permutation operator as `gsfunc`, where it contains:
- the spin component $(-1)^{s_{12}}$ * 6j coefficient * sqrt(S12 S12')
- the wigner 9j symbols with primes and unprimes * normalization (sqrt())
- summation over it
- the `gfunc`, which contains the orbital part

In [95]:
# qnalpha.append({"alpha":alpha, "alphal":alphal, "l":l12,"lam":lam, "s": s12, "s3":s3, "j":j, 
#                              "j3":j3, "bl":bl, "bs":bs, "bj":bj})

In [22]:
# bm summation then gives G 
gfunc=np.zeros((nalpha,nalpha,nqpoints,nqpoints,nx),dtype=np.double)
for qnset in qnalpha:  # go through allowed l,lam combinations
    
    
    alpha=qnset["alpha"]
    l=qnset["l"]
    lam=qnset["lam"]
    s=qnset["s"]
    s3=qnset["s3"]
    j = qnset["j"]
    j3 = qnset["j3"]
#     bl = l + lam
#     bs = s + s3
#     bj = bl + bs
#     bl = qnset["bl"]
    bs = qnset["bs"]
    bj =qnset["bj"]
    
    print(l, s, j, lam, s3, j3, bl, bs, bj)
    
    for qnsetp in qnalpha:  # go through allowed l,lam combinations
        
        # not sure how this prime stuff works here
        alphap=qnsetp["alpha"]
        sp = s
        jp = j
        j3p = j3
        
        
        
        # no need to code in the summation for L, S since they are the total orbital ang mom and spin,
        # which is fixed for each problem that we do.
        
        # evaluate wigner 9j symbol
        c9j = float(Wigner9j(l, s, j, lam, s3, j3, bl, bs, bj).doit())
        c9jp = float(Wigner9j(l, sp, jp, lam, s3, j3p, bl, bs, bj).doit())
        
        # evaluate spin component -> Wigner 6j symnbols
        c6j = float(Wigner6j(s3, 0.5, sp, s3, bs, s).doit())
        spin_part = (-1)**s * np.sqrt((2*s + 1) * (2*sp + 1)) * c6j
        
        # evaluate coefficients before that 
        coeff = (2*bs + 1) * np.sqrt((2*j+1) * (2*jp+1) * (2*j3+1) + (2*j3p + 1))
        
#         print(c9j, c9jp, c6j, spin_part, coeff)
        
        for bm in range(-bl,bl+1):
            if(abs(bm)<=l):  
                lmindx=_lmindx(l,bm) 
                orbital_part = 8*m.pi**2*np.sqrt((2*lam+1)/(4*m.pi))/(2*bl+1) \
                    *ystarl[lmindx,:,:,:]*ylylam[alphap,bm+bl,:,:,:]
                
                print(np.nonzero(ylylam[alphap,bm+bl,:,:,:]))
                
                gfunc[alpha, alphap, :,:,:] += coeff * c9j * c9jp * spin_part * orbital_part

0 0 0 1 0.5 1.5 1 0.5 1.5
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
0 1 1 1 0.5 1.5 1 1.5 2.5
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [99]:
np.nonzero(gfunc)

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))

In [23]:
gfunc[1]

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    